In [1]:
# import libraries
from gettext import npgettext
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import numpy as np
from statistics import mean, stdev

import json

In [2]:
# setup spark
spark = (
    SparkSession.builder.appName("aggregate data for first 3 final model variables")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "15g")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/29 00:04:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/29 00:04:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/29 00:04:58 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/09/29 00:04:58 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
data = spark.read.csv("../data/curated/2016_age.csv", header = True)

In [4]:
data = data.withColumnRenamed('AGEP Age','postcode')
data

22/09/29 00:05:02 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


postcode,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,Total
"2000, NSW",181,174,145,113,93,78,61,57,54,40,60,59,50,33,54,48,81,70,297,540,713,796,912,1116,1251,1257,1359,1224,1122,1050,1010,982,968,911,795,670,661,546,407,414,309,300,316,249,278,262,248,187,198,215,222,178,230,217,196,210,167,167,165,182,149,183,162,141,143,140,125,115,141,137,112,103,73,83,83,71,45,45,46,57,44,42,28,32,19,35,25,20,21,9,9,9,11,3,4,0,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27411
"2006, NSW",0,0,0,0,0,0,0,0,0,0,7,0,4,0,0,0,0,9,175,370,283,175,67,36,25,15,13,10,3,3,5,3,3,0,0,0,0,0,0,0,3,0,0,0,6,0,0,0,0,3,3,0,0,0,0,5,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1261
"2007, NSW",51,25,57,41,27,23,12,26,15,23,13,17,17,12,11,18,20,56,219,395,439,474,545,686,591,472,370,339,331,264,242,222,230,191,179,177,142,119,104,89,90,88,70,63,57,59,62,56,39,58,40,54,48,32,33,46,37,29,22,36,37,46,30,31,24,42,34,24,22,25,15,22,9,11,12,10,10,9,18,21,9,13,10,12,8,5,11,5,4,3,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8846
"2008, NSW",60,45,49,29,15,24,13,19,25,20,21,30,21,7,18,20,23,83,298,558,672,559,681,903,796,661,583,517,413,376,367,324,278,263,241,193,168,149,121,109,91,98,82,92,70,87,81,82,59,58,60,73,61,61,65,65,57,49,53,47,39,56,41,34,36,39,42,31,28,30,26,21,17,14,12,11,12,12,8,7,0,8,3,4,8,9,3,3,0,0,4,6,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11712
"2009, NSW",151,145,119,115,77,59,84,76,67,57,53,48,54,45,45,34,35,51,76,87,138,164,203,290,361,363,437,434,390,391,433,450,431,385,377,377,312,295,245,221,215,174,166,169,187,170,141,134,136,115,136,127,151,125,142,141,118,146,119,120,125,112,121,116,93,93,120,105,90,97,67,67,73,58,50,46,43,29,29,33,33,19,23,14,18,12,16,14,9,7,3,0,4,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12813
"2010, NSW",181,196,117,105,97,88,94,77,69,71,56,49,57,42,58,68,63,64,104,145,196,248,353,610,760,865,1069,1110,1090,1003,1043,1004,984,891,870,800,635,621,530,500,485,482,454,451,453,406,366,390,388,322,329,338,343,323,324,291,287,263,246,248,234,238,226,235,162,191,174,151,181,173,134,163,119,101,64,91,70,107,71,83,82,59,69,43,54,56,36,38,27,30,22,15,12,14,10,3,8,3,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27733
"2011, NSW",108,79,66,72,67,49,56,39,37,35,32,32,35,37,37,33,35,45,115,188,191,302,383,472,539,651,749,726,735,760,782,691,768,602,566,502,504,378,384,356,340,325,324,286,344,326,293,297,240,239,234,215,262,237,243,266,235,224,228,236,213,198,193,181,168,181,172,188,181,159,125,125,124,94,119,118,79,76,59,55,91,65,42,50,61,43,41,52,59,47,44,31,25,22,16,17,12,12,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21199
"2015, NSW",142,113,114,102,79,89,71,64,58,59,43,37,59,47,34,38,54,24,48,56,75,95,123,210,240,286,314,366,361,327,356,349,302,321,310,244,250,229,207,209,188,177,186,175,173,147,174,139,118,130,128,106,109,98,92,102,85,83,80,80,68,68,66,53,56,54,39,50,39,29,38,32,28,22,20,14,13,18,7,14,13,25,15,9,7,9,6,0,12,6,3,8,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9922
"2016, NSW",119,105,89,86,65,55,55,44,53,47,42,29,40,44,36,31,34,37,70,104,148,174,245,268,334,419,425,424,461,451,403,395,358,371,357,304,282,211,214,183,202,175,205,197,190,191,163,202,177,161,159,175,167,168,149,179,166,118,118,134,122,109,128,106,121,112,96,110,100,98,87,64,54,69,58,85,72,68,63,72,45,55,39,44,33,55,26,18,18,16,7,13,5,5,3,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13213
"2017, NSW",264,253,206,164,142,114,110,94,88,79,70,55,71,47,77,78,73,69,481,804,775,641,698,805,933,960,976,914,964,893,876,854,793,717,625,558,476,413,363,361,282,274,270,256,227,286,211,189,187,175,181,178,153,196,199,178,183,136,181,173,162,142,144,128,121,148,130,129,153,127,115,78,100,81,

In [14]:
data_agg = data.withColumn("18-24", sum(data[col] for col in range(18+1,24+1+1)))\
    .withColumn("25-34", sum(data[col] for col in range(25+1,35+1)))\
    .withColumn("35-44", sum(data[col] for col in range(35+1,45+1)))\
        .withColumn("45-54", sum(data[col] for col in range(45+1,55+1)))\
            .withColumn("55-64", sum(data[col] for col in range(55+1,64+1)))\
                .withColumn("65+", sum(data[col] for col in range(65+1,115+1+1)))

In [15]:
output = data_agg.select("postcode","18-24","25-34","35-44","45-54","55-64","65+")

In [16]:
# check if there is null values 
from pyspark.sql.functions import isnan, when, count, col

output.select([count(when(isnan(c), c)).alias(c) for c in output.columns]).show()

+--------+-----+-----+-----+-----+-----+---+
|postcode|18-24|25-34|35-44|45-54|55-64|65+|
+--------+-----+-----+-----+-----+-----+---+
|       0|    0|    0|    0|    0|    0|  0|
+--------+-----+-----+-----+-----+-----+---+



In [18]:
output = output.withColumn("total", sum(output[col] for col in range(1,7)))

In [28]:
output.toPandas().to_csv('../data/curated/Age_after_agg.csv', index = False)
# found that adding the total with ppl<18 cannot made up to the Total in raw data

# investigation on the raw data Total

In [26]:
# sum of numbers across rows are different from teh totals in the raw data
col_list = [str(i) for i in range(0,116)]
df2 = data.withColumn(
    'SUM',
    sum([F.col(c) for c in col_list])
)

In [27]:
df2

postcode,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,Total,SUM1
"2000, NSW",181,174,145,113,93,78,61,57,54,40,60,59,50,33,54,48,81,70,297,540,713,796,912,1116,1251,1257,1359,1224,1122,1050,1010,982,968,911,795,670,661,546,407,414,309,300,316,249,278,262,248,187,198,215,222,178,230,217,196,210,167,167,165,182,149,183,162,141,143,140,125,115,141,137,112,103,73,83,83,71,45,45,46,57,44,42,28,32,19,35,25,20,21,9,9,9,11,3,4,0,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27411,27420.0
"2006, NSW",0,0,0,0,0,0,0,0,0,0,7,0,4,0,0,0,0,9,175,370,283,175,67,36,25,15,13,10,3,3,5,3,3,0,0,0,0,0,0,0,3,0,0,0,6,0,0,0,0,3,3,0,0,0,0,5,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1261,1230.0
"2007, NSW",51,25,57,41,27,23,12,26,15,23,13,17,17,12,11,18,20,56,219,395,439,474,545,686,591,472,370,339,331,264,242,222,230,191,179,177,142,119,104,89,90,88,70,63,57,59,62,56,39,58,40,54,48,32,33,46,37,29,22,36,37,46,30,31,24,42,34,24,22,25,15,22,9,11,12,10,10,9,18,21,9,13,10,12,8,5,11,5,4,3,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8846,8840.0
"2008, NSW",60,45,49,29,15,24,13,19,25,20,21,30,21,7,18,20,23,83,298,558,672,559,681,903,796,661,583,517,413,376,367,324,278,263,241,193,168,149,121,109,91,98,82,92,70,87,81,82,59,58,60,73,61,61,65,65,57,49,53,47,39,56,41,34,36,39,42,31,28,30,26,21,17,14,12,11,12,12,8,7,0,8,3,4,8,9,3,3,0,0,4,6,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11712,11713.0
"2009, NSW",151,145,119,115,77,59,84,76,67,57,53,48,54,45,45,34,35,51,76,87,138,164,203,290,361,363,437,434,390,391,433,450,431,385,377,377,312,295,245,221,215,174,166,169,187,170,141,134,136,115,136,127,151,125,142,141,118,146,119,120,125,112,121,116,93,93,120,105,90,97,67,67,73,58,50,46,43,29,29,33,33,19,23,14,18,12,16,14,9,7,3,0,4,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12813,12849.0
"2010, NSW",181,196,117,105,97,88,94,77,69,71,56,49,57,42,58,68,63,64,104,145,196,248,353,610,760,865,1069,1110,1090,1003,1043,1004,984,891,870,800,635,621,530,500,485,482,454,451,453,406,366,390,388,322,329,338,343,323,324,291,287,263,246,248,234,238,226,235,162,191,174,151,181,173,134,163,119,101,64,91,70,107,71,83,82,59,69,43,54,56,36,38,27,30,22,15,12,14,10,3,8,3,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27733,27725.0
"2011, NSW",108,79,66,72,67,49,56,39,37,35,32,32,35,37,37,33,35,45,115,188,191,302,383,472,539,651,749,726,735,760,782,691,768,602,566,502,504,378,384,356,340,325,324,286,344,326,293,297,240,239,234,215,262,237,243,266,235,224,228,236,213,198,193,181,168,181,172,188,181,159,125,125,124,94,119,118,79,76,59,55,91,65,42,50,61,43,41,52,59,47,44,31,25,22,16,17,12,12,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21199,21174.0
"2015, NSW",142,113,114,102,79,89,71,64,58,59,43,37,59,47,34,38,54,24,48,56,75,95,123,210,240,286,314,366,361,327,356,349,302,321,310,244,250,229,207,209,188,177,186,175,173,147,174,139,118,130,128,106,109,98,92,102,85,83,80,80,68,68,66,53,56,54,39,50,39,29,38,32,28,22,20,14,13,18,7,14,13,25,15,9,7,9,6,0,12,6,3,8,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9922,9920.0
"2016, NSW",119,105,89,86,65,55,55,44,53,47,42,29,40,44,36,31,34,37,70,104,148,174,245,268,334,419,425,424,461,451,403,395,358,371,357,304,282,211,214,183,202,175,205,197,190,191,163,202,177,161,159,175,167,168,149,179,166,118,118,134,122,109,128,106,121,112,96,110,100,98,87,64,54,69,58,85,72,68,63,72,45,55,39,44,33,55,26,18,18,16,7,13,5,5,3,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13213,13188.0
"2017, NSW",264,253,206,164,142,114,110,94,88,79,70,55,71,47,77,78,73,69,481,804,775,641,698,805,933,960,976,914,964,893,876,854,793,717,625,558,476,413,363,361,282,274,270,256,227,286,211,189,187,175,181,178,153,196,199,